In [ ]:
from glob import glob
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from skimage import io, transform
import pywt
import os
import cv2
import random

In [ ]:
from tensorflow.keras.layers import (
    Input,Dense,Reshape,Flatten,Dropout,Concatenate,Softmax)
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D,Conv2DTranspose
from tensorflow.keras.models import  Model
from tensorflow.keras.optimizers import Adam
import tensorflow 
from tensorflow.keras.layers import Lambda, ReLU
from tensorflow.keras.losses import logcosh
from tensorflow.keras.models import Sequential

In [1]:
import dumpman

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" #this is to set the GPU to be used
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #if we have multiple GPUs, we can use 0,1 as well. Comment if no GPU available

In [ ]:
X = dumpman.undumper('list1.pkl') #list of file names for input
y = dumpman.undumper('list2.pkl') #list of file names for Ground Truth

X_train, X_rem, y_train, y_rem = train_test_split(X, y, test_size=0.05, random_state=42)


X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size=0.5, random_state=42)

In [ ]:
print(len(X_train))
print(len(X_valid))

In [ ]:
fname = './keras'

batch = 4 #define the batch size. Higher batch size needs higher RAM as this amount of data needs to be on RAM every iteration
num_epochs = 35 #this controls how many epochs to run the network for training, before stopping

In [ ]:
img_row = 1024 #change these values to the size you need for your network
img_col = 1024 #network size depends on this as well
channels = 1
img_shape = (img_row,img_col,channels)

In [ ]:
use_cuda = True

In [ ]:
def encode_same(inputs,filters,kernel=3,bn=True,drop=True): #function that returns
    d=Conv2D(filters=filters,kernel_size=kernel,strides=1,padding='same')(inputs)
    d=ReLU()(d)
    if bn:
        d= BatchNormalization()(d)
    if drop:
        d = Dropout(0.5)(d)
    return d

In [ ]:
def encode_half(inputs,filters,kernel=3,bn=True,drop=True): #function that returns
    d=Conv2D(filters=filters,kernel_size=kernel,strides=2,padding='same')(inputs)
    d=ReLU()(d)
    if bn:
        d= BatchNormalization()(d)
    if drop:
        d = Dropout(0.5)(d)
    return d

In [ ]:
def generator(): #class definition for the network
    
    inputs=Input(shape=(img_row,img_col,channels))
    
    inp = encode_same(inputs,1,bn=False,drop=False)

    e = encode_half(inp,3) #half the shape

    op = Conv2D(3, (3, 3), strides =(1,1), padding="same", activation = "sigmoid")(e) 
    
    return Model(inputs,op)

In [ ]:
G=generator() #creates a variable for the DL network

In [ ]:
G.summary() #displays the network

In [ ]:
optimizer=Adam(0.1) #create an optimizer. We use Adam

img = Input(shape=img_shape) #define the input shape variable
op = Input(shape=(img_row/2,img_col/2,channels)) #define the output shape variable

G.compile(loss=['mse'], optimizer=optimizer) #compile our network with initialization parameters

In [ ]:
def lr_sch(epoch, lr): #this will control the learning rate. Every 3 epochs, make the LR 10% of previous value
    decay_rate = 0.1
    decay_step = 3
    if epoch % decay_step == 0 and epoch:
        return lr * decay_rate
    return lr
    
lr_scheduler = tensorflow.keras.callbacks.LearningRateScheduler(lr_sch) #integrate the LR function to work in Keras scheduler

In [ ]:
lr_reducer = tensorflow.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, mode='min') #reduce the LR depending on validation loss
checkpointer = tensorflow.keras.callbacks.ModelCheckpoint(filepath=fname, verbose=1, monitor='loss', mode='min',
                                               save_weights_only=True, save_best_only=True) #save network based on training loss

In [ ]:
class My_Generator(tensorflow.keras.utils.Sequence): #this will be our training generator, sub classed from tensorflow.keras.utils.Sequence

    def __init__(self, input_filenames, output_filenames, batch_size= batch):
        self.input_filenames, self.output_filenames = input_filenames, output_filenames #this copies the values to local variables for our immediate use
        self.batch_size = batch_size
        
    def __len__(self):
        return np.uint16(np.ceil(len(self.input_filenames) / float(self.batch_size))) #since the number of input and output files are same, we can use either

    def __getitem__(self, idx):
        batch_x = self.input_filenames[idx * self.batch_size:(idx + 1) * self.batch_size] #creates the batches of file names
        batch_y = self.output_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        inp = np.zeros((len(batch_x),img_row,img_col,1),dtype=np.float32) #we go from a 1 layer input, like a grayscale image        
        op = np.zeros((len(batch_y),img_row/2,img_col/2,3),dtype=np.float32) #to a 3 layer output like RGB, for something like colorization
            
        for i in range(len(batch_x)): 
            inp[i,:,:,0] = cv2.resize(cv2.imread(batch_x[i], 0),(img_col,img_row)) #note that unlike PyTorch, the last dimension is 0
            #here we read the file name in grayscale (0 flag in opencv), then resize it to our desired shape for DL
            #and then copy the values into our blank input array
            
            op[i] = cv2.resize(cv2.imread(batch_y[i]),(img_col/2,img_row/2)) #we do not need to reshape unlike PyTorch
            #we do the same thing here, but since the files are 3 layered, we resize and then reshape since 
            #opencv has the shape as col,row,3 while PyTorch uses 3,col,row
        
        return (inp/255, op/255) #normalize to 0-1 values for our use case

In [ ]:
class My_val_Generator(tensorflow.keras.utils.Sequence): #validation generator with the same things. But different name
    
    def __init__(self, input_filenames, output_filenames, batch_size= batch):
        self.input_filenames, self.output_filenames = input_filenames, output_filenames #this copies the values to local variables for our immediate use
        self.batch_size = batch_size
        
    def __len__(self):
        return np.uint16(np.ceil(len(self.input_filenames) / float(self.batch_size))) #since the number of input and output files are same, we can use either

    def __getitem__(self, idx):
        batch_x = self.input_filenames[idx * self.batch_size:(idx + 1) * self.batch_size] #creates the batches of file names
        batch_y = self.output_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        inp = np.zeros((len(batch_x),1,img_row,img_col),dtype=np.float32) #we go from a 1 layer input, like a grayscale image        
        op = np.zeros((len(batch_y),3,img_row/2,img_col/2),dtype=np.float32) #to a 3 layer output like RGB, for something like colorization
            
        for i in range(len(batch_x)): 
            inp[i,:,:,0] = cv2.resize(cv2.imread(batch_x[i], 0),(img_col,img_row)),(img_col,img_row)
            #here we read the file name in grayscale (0 flag in opencv), then resize it to our desired shape for DL
            #and then copy the values into our blank input array
            
            op[i] = np.reshape(cv2.resize(cv2.imread(batch_y[i]),(img_col/2,img_row/2)),(3,img_col/2,img_row/2))
            #we do the same thing here, but since the files are 3 layered, we resize and then reshape since 
            #opencv has the shape as col,row,3 while PyTorch uses 3,col,row
        
        return (inp/255, op/255) #normalize to 0-1 values for our use case

In [ ]:
my_training_batch_generator = My_Generator(X_train, y_train, batch_size= batch) #create function for training generator
my_validation_batch_generator = My_val_Generator(X_valid, y_valid, batch_size= batch) #create function for validation generator

In [ ]:
hist = G.fit(my_training_batch_generator, steps_per_epoch=(len(X_train) // batch),                                          
                                          epochs=num_epochs,
                                          callbacks=[lr_reducer, lr_scheduler, checkpointer],
                                          verbose=1,
                                          validation_data=my_validation_batch_generator,
                                          validation_steps=(len(X_valid) // batch)) 
#this will take care of the training, including saving, based on the different LR variables we have created beforehand

# G.save_weights(fname1, overwrite=True) #we can use this at the end to save the last epoch, but rename it just in case it is not the best output

In [ ]:
img_list = X_test #this is our training list

G.load_weights(fname)  #load the trained weights                

for i, img in enumerate(img_list): #go through each value in list
    
    name = img[46:len(img)-4] #get the name. Experiment and get the exact name to save against
    # print(name)
    
    inp = cv2.imread(img, 0) #make the input image as grayscale
    temp = np.array(inp)
    
    cv2.imwrite('./data/test/'+(name) + '_ip.png', inp) #write the image to disk
    
    shape0 = temp.shape[0] #calculate the shapes for reshaping later
    shape1 = temp.shape[1]
    
    inp = cv2.resize(inp, (img_col, img_row))   #resize the input to our input size
    
    d = np.reshape(inp, (1, 1, img_col, img_row) #make it a 4D tensor
    
    d = d.astype('float') #go to float

    k = G.predict(d) #run the model and get the output
    
    op = cv2.resize(k[0]*255, (shape0/2, shape1/2)) #resize the output to our actual size/2 after normalization back to 0-255
    
    cv2.imwrite('./data/test/'+(name) + '_op.jpg', op) #save the op. No need to reshape as Keras outputs as channels last. This is a channel last output